#read the readme to know exactly what code is doing. 

Here is the breakdown of how I want to build the code. 

- We will use requests to load the first page of the results. 
- Store the links to each job description. I don't want to use the in page feature of the job description. That makes the HTML too complicated.
- Once we have the links, use requests to get content for each one. 
- Use beutiful soup to scrape the job description. 
- The idea is to give the user the choice of skills he wants the summary. For the initial process I will provide a list of skills and check the results to see if everything works clearly. Then move on to user inputs. 
- Once the code works for let's say 10 or 20 job searches, we can scale it to incorporate more companies.
- Be ethical. I will keep a cap on how many companies we can search in the end. Let's not be scummy and inundate the indeed server.


In [1]:
#let's begin 

import pyforest 
import requests 
from bs4 import BeautifulSoup

In [2]:
job_page = requests.get('https://www.indeed.com/jobs?q=data+scientist&l=United+States')
job_page

<Response [200]>

In [3]:
first_page_content = job_page.text

In [4]:
parser = BeautifulSoup(first_page_content, 'html.parser')

In [5]:
test = parser.body.find('div', attrs = {'data-tn-component':'organicJob'})

In [6]:
test_job_title = test.a.text.strip()
test_job_title

'Data Scientist'

In [7]:
job_link = test.a.get('href')

In [8]:
job_link

'/company/WTF-DIGITAL/jobs/Data-Scientist-dda20b6589d3de4f?fccid=8020b1127d57b812&vjs=3'

So we pretty much figured out how to get the job name and link. 

In [9]:
company_name = test.div.span.text.strip()
company_name

'dynamicbitit.com'

Now let's check if the same code can work for the rest of the job listings. 

In [10]:
test2 = parser.body.find_all('div', attrs = {'data-tn-component':'organicJob'})

In [11]:
len(test2)

14

In [12]:
test2[1].a.text.strip()

'Sr. Data Scientist'

In [13]:
test2[3].a.text.strip()

'Research Data Scientist II'

In [14]:
test2[6].a.text.strip()

'Data Scientist - Analytics'

Works so far. One issue that is arising is not every job has the div-tn-component attribute. We can choose to ignore this since we will be looking at quite a number of jobs. Indeed has 15 job listings per page. As of this code some jobs are being scraped and some aren't. I'm continuing and will come back to see if I can change something. Ok let put this in a loop and see if it works. 

In [15]:
!git status 

On branch master
Your branch is up to date with 'origin/master'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   code_tester.ipynb

no changes added to commit (use "git add" and/or "git commit -a")


In [16]:
!git add code_tester.ipynb

The file will have its original line endings in your working directory


In [17]:
!git commit -m "Code for name, company and links to work on single instances."

[master 6b65ce9] Code for name, company and links to work on single instances.
 1 file changed, 111 insertions(+), 22 deletions(-)


In [18]:
!git push

To https://github.com/hamzajabbarr/skills-summariser.git
   1a81ec5..6b65ce9  master -> master


____________________________________________________________________________________________

In [19]:
job_table = pd.DataFrame(columns = ['Job Title','Organization','Link'])

<IPython.core.display.Javascript object>

In [20]:
job_table

,Job Title,Organization,Link


In [21]:
scraped_page = parser.body.find_all('div', attrs = {'data-tn-component':'organicJob'})

In [ ]:
for job in scraped_page:
    
    title = job.a.text.strip()
    org = job.div.span.text.strip()
    link = job.a.get('href')
    
    job_table['Job Title'] = title
    job_table['Organization'] = org
    
    if link.startswith('https'):
        job_table['Link'] = link
    else:
        join_link = "https://www.indeed.com"